<a href="https://colab.research.google.com/github/DIPANJAN001/Dipanjanpatra/blob/master/SolarManagementAgent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

import numpy as np
import matplotlib.pyplot as plt
import os
from collections import deque
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam
import random

In [4]:
import numpy as np
import gym
from gym.utils import seeding
from gym.spaces import Space, Discrete, MultiDiscrete,  Box
from gym.spaces.space import Space
import numpy as np
import random
from collections import deque
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam
from keras import metrics

In [10]:
class MyEnv3(gym.Env):
    
      def __init__(self):
        
        self.action_space = Discrete(4)
        self.observation_space=None
        #self.battery_efficiency=0.3
        self.battery_cap=30
        self.max_battery_discharge=15
        self.max_battery_charge=10
        self.max_battery_capacity=50
        #self.reward, self.done, self.info= None, None, None
        #self.grid_import=0
        #self.grid_export=0
        self.info=None
      def _seed(self, seed=None):
        self.np_random, seed = seeding.np_random(seed)
        return [seed]

      def _step(self,action):
        grid_import_=0
        grid_export_=0
        done=False
        load=np.random.randint(100,150)
        pv=np.random.randint(80,130)
        battery_state=self.battery_cap
        #net_load=max((load-power),0)
        #net_rem=max(net_load-(self.battery_efficiency*self.battery_cap),0)
        info={}
        reward=0
        #battery_chng=0
        
        if action== 0:
          grid_export_= max(pv-load-self.max_battery_charge,0) # grid export
          self.battery_cap= max(min(self.battery_cap+min(self.max_battery_charge,pv-load),50),0)
          grid_import_=0
          info={
              'grid_export':grid_export_,
              'grid_import':grid_import_
          }
        if action== 1:
          grid_export_= max(pv-load-self.max_battery_charge,0) # grid export
          self.battery_cap= max(min(self.battery_cap+min(self.max_battery_charge,pv-load),50),0)
          grid_import_=0
          # only battery charge
          info={
              'grid_export':grid_export_,
              'grid_import':grid_import_
          }
        if action== 2:
            grid_export_=0
            self.battery_cap= min(max(self.battery_cap-self.max_battery_discharge,0),50)# battery discharge
            grid_import_=max(load-pv-self.max_battery_discharge,0)
            info={
              'grid_export':grid_export_,
              'grid_import':grid_import_
             }
        if action== 3:
          grid_export_=0
          self.battery_cap= min(max(self.battery_cap-min(self.max_battery_discharge,load-pv),0),50)#change in battery
          grid_import_=max(load-pv-self.max_battery_discharge,0)
         #only battery discharge
          info={
              'grid_export':grid_export_,
              'grid_import':grid_import_
                    }     
        
        reward=grid_export_*8-grid_import_*10
        
        if(self.battery_cap<7.0):
              done=True
      
        return battery_state,load,pv,reward,self.battery_cap, done,info
        
      def _reset(self):
        self.battery_cap=30
        #self.done=False
        return self.battery_cap

In [11]:
env=MyEnv3()

In [7]:
class Agent:
    def __init__(self,state_size,action_size):
        self.state_size = state_size
        self.action_size = action_size
        self.memory = deque(maxlen=2000)
        self.gamma = 0.95 #Discount Factor
        self.epsilon = 1.0 # Exploration Rate: How much to act randomly, 
        self.epsilon_decay = 0.995
        self.epsilon_min = 0.01
        self.learning_rate = 0.001 
        self.model = self._create_model()
        
    
    def _create_model(self):
        #Neural Network To Approximate Q-Value function
        model = Sequential()
        model.add(Dense(24,input_dim=self.state_size,activation='relu')) #1st Hidden Layer
        model.add(Dense(24,activation='relu')) #2nd Hidden Layer
        model.add(Dense(self.action_size,activation='linear'))
        model.compile(loss='mse',optimizer=Adam(lr=self.learning_rate))
        return model
    
    def remember(self,state,action,reward,next_state,done):
        self.memory.append((state,action,reward,next_state,done)) #remembering previous experiences
        
    def act(self,state):
        # Exploration vs Exploitation
        if np.random.rand()<=self.epsilon:
            return random.randrange(self.action_size)
        act_values = self.model.predict(state) # predict reward value based upon current state
        return np.argmax(act_values[0]) #Left or Right
    
    def train(self,batch_size=32): #method that trains NN with experiences sampled from memory
        minibatch = random.sample(self.memory,batch_size)
        for state,action,reward,next_state,done in minibatch:
            
            if not done: #boolean 
                target = reward + self.gamma*np.amax(self.model.predict(next_state)[0])
            else:
                target = reward
            target_f = self.model.predict(state)
            target_f[0][action] = target
            self.model.fit(state,target_f,epochs=1,verbose=0) #single epoch, x =state, y = target_f, loss--> target_f - 
            
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay
    
    def load(self,name):
        self.model.load_weights(name)
    def save(self,name):
        self.model.save_weights(name)

In [8]:
agent = Agent(state_size=3,action_size=4)
done = False
state_size = 3
action_size =4
batch_size = 32

/usr/local/lib/python3.8/dist-packages/keras/optimizers/optimizer_v2/adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [ ]:
agent = Agent(state_size, action_size) # initialise agent
done = False
for e in range(n_episodes):
    state = env.reset()
    state = np.reshape(state,[1,state_size])
    
    for time in range(5000):
        env.render()
        action = agent.act(state) #action is 0 or 1
        next_state,reward,done,other_info = env.step(action) 
        reward = reward if not done else -10
        next_state = np.reshape(next_state,[1,state_size])
        agent.remember(state,action,reward,next_state,done)
        state = next_state
        
        if done:
            print("Game Episode :{}/{}, High Score:{},Exploration Rate:{:.2}".format(e,n_episodes,time,agent.epsilon))
            break
            
    if len(agent.memory)>batch_size:
        agent.train(batch_size)
    
    if e%50==0:
        agent.save(output_dir+"weights_"+'{:04d}'.format(e)+".hdf5")
        
env.close()

In [18]:
for e in range(400): #Episode
    #Play 20 episodes 
    score = 0
    observation = env._reset()
    #print(observation)
    done=False
    while not done:
        #env.render()
        action = env.action_space.sample()
        print(action)
        observation1,observation2,observation3,observation4,reward,done,info = env._step(action)
        print(observation1,observation2,observation3,observation4)
        print(info)
        score+=round(reward)
        
        #print(info['load'],info['power'])
      
    print('episode:{} Profit:{}'.format(e,score))

3
30 106 95 0
{'grid_export': 0, 'grid_import': 0}
0
19 126 123 0
{'grid_export': 0, 'grid_import': 0}
1
16 149 90 0
{'grid_export': 0, 'grid_import': 0}
episode:0 Profit:35
0
30 118 96 0
{'grid_export': 0, 'grid_import': 0}
2
8 106 116 0
{'grid_export': 0, 'grid_import': 0}
episode:1 Profit:8
0
30 127 96 0
{'grid_export': 0, 'grid_import': 0}
episode:2 Profit:0
1
30 106 115 0
{'grid_export': 0, 'grid_import': 0}
0
39 113 120 0
{'grid_export': 0, 'grid_import': 0}
3
46 137 85 -370
{'grid_export': 0, 'grid_import': 37}
2
31 148 101 -320
{'grid_export': 0, 'grid_import': 32}
2
16 125 103 -70
{'grid_export': 0, 'grid_import': 7}
episode:3 Profit:133
3
30 134 82 -370
{'grid_export': 0, 'grid_import': 37}
3
15 100 83 -20
{'grid_export': 0, 'grid_import': 2}
episode:4 Profit:15
3
30 107 93 0
{'grid_export': 0, 'grid_import': 0}
2
16 121 120 0
{'grid_export': 0, 'grid_import': 0}
episode:5 Profit:17
0
30 144 124 0
{'grid_export': 0, 'grid_import': 0}
0
10 114 117 0
{'grid_export': 0, 'grid_im